In [1]:
# 统计一阶邻居分类个数
import dgl
import os 
from multiprocessing import Pool
import pickle
import torch
import numpy as np

[10:26:44] Using backend: pytorch
/opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: libtorch_cuda_cpp.so: cannot open shared object file: No such file or directory


In [2]:
base_path='./dataset'
graphs, _ = dgl.load_graphs(os.path.join(base_path, 'graph.bin'))
graph = graphs[0]
# graph = dgl.add_self_loop(graph)

In [3]:
with open(os.path.join(base_path, 'k_fold_labels.pkl'), 'rb') as f:
    label_data = pickle.load(f)
    labels = label_data['label']

In [4]:
# test_num = 2000
test_num = graphs[0].num_nodes()
nids = range(test_num)

In [6]:
def count_map(nid):
    g_out = dgl.sampling.sample_neighbors(graph, [nid], -1, edge_dir='out')
    g_in = dgl.sampling.sample_neighbors(graph, [nid], -1, edge_dir='in')
    src_out, dst_out = g_out.all_edges()
    src_in, dst_in = g_in.all_edges()
    # ids = torch.cat((dst_out, src_in), dim=-1).numpy()
    # ids = list(set(ids))
    in_labs = labels[src_in].reshape((-1))
    out_labs = labels[dst_out].reshape((-1))
    print(nid, dst_out, src_in)
    count_in = np.zeros(24)
    count_out = np.zeros(24)
    for i in in_labs:
        if i == -1:
            count_in[23]+=1
        else:
            count_in[i]+=1
    for i in out_labs:
        if i == -1:
            count_out[23]+=1
        else:
            count_out[i]+=1
    return [count_in, count_out]

In [9]:
count_map(10)

10 tensor([ 364306, 1739031, 2149841, 1579857,  953930, 2574863,  434964, 3020196,
         348955,  890230, 1616426, 1535164, 1957584,  951120, 1722732,  420833,
         219955]) tensor([2038534, 2234690, 3099699, 1833938,  845227, 1642848, 1024894])


[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 6.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 15.])]

In [33]:
result = []
for nid in nids:
    result.append(count_map(nid))
result = np.array(result)
np.save(os.path.join(base_path, 'neighbors_class_count.npy'), result)
result.shape

(3655452, 2, 24)

In [5]:
base_path='./dataset'

In [6]:
import pandas as pd
import numpy as np
result = np.load(os.path.join(base_path, 'neighbors_class_count.npy'))
result.shape

(3655452, 2, 24)

In [20]:
result1 = np.sum(result,axis=-2)
result1[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [25]:
result2 = np.sum(result1,axis=-1)
index2 = np.argwhere(result2>0)
index2.shape

(3031367, 1)

In [26]:
result3 = result[index2]

In [34]:
result3[31]

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 2., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 6.]]])

In [42]:
result_1 = result.reshape((-1, 48))
result_1.shape

(3655452, 48)

In [47]:
df = pd.DataFrame(result_1, columns=list(range(24))+list(range(24)))

In [49]:
desc = df.describe()

In [55]:
desc

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,...,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06,3.655452e+06
mean,8.020349e-03,1.160538e-01,2.088226e-01,2.438322e-01,1.441324e-01,1.055664e-01,6.190124e-02,1.279530e-01,5.393341e-02,4.530548e-02,...,4.107098e-02,9.471469e-02,7.206687e-02,7.563989e-02,3.997727e-02,4.088167e-02,9.794630e-02,1.455694e-01,4.092025e-02,4.873557e+00
std,1.843208e-01,1.721802e+00,2.537638e+00,3.935248e+00,3.501663e+00,1.519428e+00,6.730180e-01,1.063501e+00,9.311184e-01,5.110139e-01,...,3.706920e-01,6.977154e-01,6.089858e-01,4.690872e-01,5.385749e-01,3.796552e-01,7.544427e-01,8.290626e-01,3.603732e-01,7.149208e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00
max,8.000000e+01,1.051000e+03,1.390000e+03,1.693000e+03,2.402000e+03,5.280000e+02,2.250000e+02,3.010000e+02,2.300000e+02,2.030000e+02,...,6.900000e+01,1.350000e+02,7.900000e+01,5.500000e+01,2.020000e+02,5.900000e+01,5.200000e+01,1.900000e+02,4.500000e+01,3.769000e+03


In [57]:
desc.iloc[-1]

0        80.0
1      1051.0
2      1390.0
3      1693.0
4      2402.0
5       528.0
6       225.0
7       301.0
8       230.0
9       203.0
10      546.0
11      704.0
12     1361.0
13      485.0
14      710.0
15      651.0
16      264.0
17      443.0
18      332.0
19      385.0
20      791.0
21      678.0
22      160.0
23    10117.0
0        17.0
1       146.0
2       247.0
3       152.0
4      2873.0
5       178.0
6        77.0
7       115.0
8       324.0
9        84.0
10       75.0
11      188.0
12     1756.0
13      119.0
14       69.0
15      135.0
16       79.0
17       55.0
18      202.0
19       59.0
20       52.0
21      190.0
22       45.0
23     3769.0
Name: max, dtype: float64

In [59]:
desc.iloc[1]

0     0.008020
1     0.116054
2     0.208823
3     0.243832
4     0.144132
5     0.105566
6     0.061901
7     0.127953
8     0.053933
9     0.045305
10    0.058646
11    0.116017
12    0.202537
13    0.137903
14    0.048150
15    0.096605
16    0.038697
17    0.089519
18    0.030788
19    0.048390
20    0.071948
21    0.102323
22    0.033779
23    5.788669
0     0.011044
1     0.166942
2     0.270420
3     0.327879
4     0.252929
5     0.165723
6     0.066675
7     0.258636
8     0.077806
9     0.049010
10    0.065929
11    0.216180
12    0.359733
13    0.168241
14    0.041071
15    0.094715
16    0.072067
17    0.075640
18    0.039977
19    0.040882
20    0.097946
21    0.145569
22    0.040920
23    4.873557
Name: mean, dtype: float64

In [60]:
normalized_df=(df-df.min())/(df.max()-df.min())

In [61]:
normalized_df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000796
2,0.0,0.000951,0.0,0.0,0.0,0.0,0.004444,0.0,0.004348,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002123
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001857


In [62]:
normalized_np = normalized_df.to_numpy()

In [1]:
normalized_np[1]

NameError: name 'normalized_np' is not defined

In [36]:
normalized_np = np.load('dataset/normalized_neighbors_class_count.npy')

In [37]:
normalized_np_2 = normalized_np[index2]

In [39]:
normalized_np_2 = normalized_np_2.reshape((-1,2,24))

In [40]:
normalized_np_2.shape

(3031367, 2, 24)

In [41]:
normalized_np_2[35]

array([[0.        , 0.00285442, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00059306],
       [0.        , 0.        , 0.        , 0.        , 0.00034807,
        0.        , 0.01298701, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00079597]])